In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_3649/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_5_9_12,0.918990,0.749260,0.842403,0.816243,0.015155,0.019672,0.032106,0.025523,"Hidden Size=[45, 30], regularizer=0.02, learni..."
1,model_5_9_11,0.918981,0.770562,0.843578,0.823422,0.015157,0.018000,0.031867,0.024526,"Hidden Size=[45, 30], regularizer=0.02, learni..."
3,model_5_9_10,0.918795,0.790329,0.844709,0.830114,0.015192,0.016450,0.031636,0.023596,"Hidden Size=[45, 30], regularizer=0.02, learni..."
5,model_5_9_9,0.918424,0.808473,0.845796,0.836290,0.015261,0.015026,0.031415,0.022739,"Hidden Size=[45, 30], regularizer=0.02, learni..."
7,model_5_9_8,0.917859,0.824916,0.846846,0.841932,0.015367,0.013736,0.031201,0.021955,"Hidden Size=[45, 30], regularizer=0.02, learni..."
9,model_5_9_7,0.917100,0.839642,0.847859,0.847034,0.015509,0.012581,0.030995,0.021246,"Hidden Size=[45, 30], regularizer=0.02, learni..."
11,model_5_9_6,0.916140,0.852645,0.848835,0.851598,0.015688,0.011561,0.030796,0.020612,"Hidden Size=[45, 30], regularizer=0.02, learni..."
13,model_5_9_5,0.914979,0.863966,0.849762,0.855623,0.015906,0.010672,0.030607,0.020053,"Hidden Size=[45, 30], regularizer=0.02, learni..."
16,model_5_9_4,0.913612,0.873663,0.850622,0.859116,0.016161,0.009912,0.030432,0.019568,"Hidden Size=[45, 30], regularizer=0.02, learni..."
18,model_5_9_3,0.912041,0.881846,0.851386,0.862090,0.016455,0.009270,0.030276,0.019155,"Hidden Size=[45, 30], regularizer=0.02, learni..."
